# Cleanup resources


**This notebook ensures you delete all AWS resources that was created as part of this workshop**

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`

---

## Contents

1. [Setup](#Setup)
1. [Collect feature groups to delete](#Collect-feature-groups-to-delete)
1. [Delete feature groups and purge objects in Offline feature store](#Delete-feature-groups-and-purge-objects-in-Offline-feature-store)
1. [Cleanup other S3 artifacts created as part of workshop](#Cleanup-other-S3-artifacts-created-as-part-of-workshop)
1. [Delete SageMaker model endpoint ](#Delete-SageMaker-model-endpoint)

# Setup

In [ ]:
import sagemaker
import logging
import boto3

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [ ]:
boto_session = boto3.Session()
region = boto_session.region_name
account_id = boto3.client('sts').get_caller_identity()['Account']
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
s3 = boto3.resource('s3')

sagemaker_session = sagemaker.Session()
default_bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker-feature-store'

# Collect feature groups to delete

List and paginate feature groups to collect the ones created during this workshop

In [ ]:
# Dict to store feature groups created by this workshop
feature_groups_to_delete = []

# Retrieve list of Feature Groups using filter subword ('fscw')
next_token = None
filter_subword = 'fscw'

response = sagemaker_client.list_feature_groups(NameContains=filter_subword)
# Collect Feature Group names from returned summaries
fg_summaries = response['FeatureGroupSummaries']
for summary in fg_summaries:
    #print('#1> Appending> '+ summary['FeatureGroupName'])
    feature_groups_to_delete.append(summary['FeatureGroupName'])

# Capture NextToken from response, if it exists, for pagination
if 'NextToken' in (response.keys()):
    next_token = response['NextToken']


while next_token:
    # Retrieve next page of feature groups based on NextToken
    response = sagemaker_client.list_feature_groups(NameContains=filter_subword, NextToken=next_token)
    # Collect Feature Group names from returned summaries
    fg_summaries = response['FeatureGroupSummaries']
    for summary in fg_summaries:
        #print('#2> Appending> '+ summary['FeatureGroupName'])
        feature_groups_to_delete.append(summary['FeatureGroupName'])

    # Capture NextToken from response, if it exists, for pagination
    next_token = None
    if 'NextToken' in (response.keys()):
        next_token = response['NextToken']


feature_groups_to_delete

# Delete feature groups and purge objects in Offline feature store

In [ ]:
def delete_offline_store(feature_group_name: str):
    try:
        response = sagemaker_client.describe_feature_group(FeatureGroupName=feature_group_name)
        offline_store_config = response['OfflineStoreConfig']
    except Exception:
        print(response)
        logger.error(f'Feature group: {feature_group_name} does NOT have an offline store!')
        has_offline_store = False
        return
    
    offline_store_s3_uri = offline_store_config['S3StorageConfig']['S3Uri']
    offline_store_bucket = offline_store_s3_uri.split('s3://')[1].split('/')[0]
    offline_store_prefix = '/'.join(offline_store_s3_uri.split('/')[3:])
    full_prefix = f'{offline_store_prefix}/{account_id}/sagemaker/{region}/offline-store/{feature_group_name}'
    
    if full_prefix[0] == '/':
        full_prefix = full_prefix[1:]
        
    bucket = s3.Bucket(offline_store_bucket)
    filtered_objects = bucket.objects.filter(Prefix=full_prefix)
    
    object_count = 0
    n_objects = len(list(filtered_objects))
    logger.warning(f'You are about to delete {n_objects} objects in prefix: {full_prefix} in bucket: {offline_store_bucket}')
    logger.warning(f'Are you sure you want to these objects ? (y/n)')
    choice = input()
    if choice == 'y':
        filtered_objects.delete()

In [ ]:
def interactive_cleanup(feature_group_names: list):
    for feature_group_name in feature_group_names:
        logger.info(f'Delete feature group {feature_group_name} ? (y/n)')
        choice = input()
        if choice == 'y':
            logger.info(f'Delete offline store for {feature_group_name} ? (y/n)')
            deletion_choice = input()
            if deletion_choice == 'y':
                delete_offline_store(feature_group_name)
            logger.info(f'Deleting feature group: {feature_group_name} ...')
            sagemaker_client.delete_feature_group(FeatureGroupName=feature_group_name)
            logger.info(f'Deleted feature group: {feature_group_name}')

In [ ]:
interactive_cleanup(feature_groups_to_delete)

# Cleanup other S3 artifacts created as part of workshop

In [ ]:
!aws s3 rm s3://{default_bucket}/{prefix}/fscw/ --recursive

In [ ]:
!aws s3 rm s3://{default_bucket}/{prefix}/partitions/ --recursive

# Delete SageMaker model endpoint

In [ ]:
%store -r endpoint_name

# retrieving endpoint name created in module 3
model_endpoint_name = endpoint_name 

In [ ]:
response = sagemaker_client.describe_endpoint_config(EndpointConfigName=model_endpoint_name)
model_name = response['ProductionVariants'][0]['ModelName']
model_name

In [ ]:
sagemaker_client.delete_model(ModelName=model_name)    

In [ ]:
sagemaker_client.delete_endpoint(EndpointName=model_endpoint_name)

In [ ]:
sagemaker_client.delete_endpoint_config(EndpointConfigName=model_endpoint_name)